In [1]:
import model as m
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=256)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-04-11 18:01:54.684770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 18:01:54.692233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 18:01:54.692469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 18:01:54.693807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 18:01:54.694044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [2]:
model= m.NN_Model()
model.get_data("laser_log/datasetnowall_unidir.csv")

beta=[0.0]
drop=[0.6]
epochs=3000
batch_size=16

group_epochs=1

update=False

results=[]
min_pos_error=4.0

for i in beta:
    for j in drop:
        #with sess.as_default():
            model.model_define(drop_rate=j)
            model.set_beta(i)
            model.model_compile("adam",model.pos_loss,['mean_absolute_error'])

            for step in range(epochs//group_epochs):

                if step %5 == 0:
                    print("Epoch: ",step)

                history = model.model_run(group_epochs,batch_size, verbose=0)
                #pos_mse,rot_mse,pos_mae,rot_mae = model.get_error(verbose=1)
                pos_mse,pos_mae = model.get_error(verbose=1)
                #results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "rotation squared error":rot_mse, "position absolute error":pos_mae, "rotation absolute error":rot_mae})  
                results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "position absolute error":pos_mae})  

                if pos_mae < min_pos_error:
                    model.model_cnn.save_weights("models/cp-epoch-train{}{}.ckpt".format(i,j))
                    min_pos_error=pos_mae
                    update = True

print(update)

Epoch:  0


2022-04-11 18:04:18.476898: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


position mean square error: 7.380566576500625 -- position mean absolute error: 2.0036796522803617 


2022-04-11 18:04:32.649588: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-04-11 18:04:32.770604: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-04-11 18:04:32.890586: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


position mean square error: 8.72716137351804 -- position mean absolute error: 2.2637333775281903 
position mean square error: 5.515937691144037 -- position mean absolute error: 1.7157957674428834 


2022-04-11 18:04:58.826567: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-04-11 18:04:58.946285: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


position mean square error: 3.3762175277780653 -- position mean absolute error: 1.2655464790575222 
position mean square error: 3.847016584259462 -- position mean absolute error: 1.4524503710448755 
Epoch:  5
position mean square error: 4.488978685680057 -- position mean absolute error: 1.596968910035492 
position mean square error: 3.0118364833959967 -- position mean absolute error: 1.2709863863646995 
position mean square error: 2.786383041340195 -- position mean absolute error: 1.1792948445200928 
position mean square error: 2.9840458653256645 -- position mean absolute error: 1.249333748093249 
position mean square error: 2.3193433952724765 -- position mean absolute error: 1.0290337768465285 
Epoch:  10
position mean square error: 2.7852535472102162 -- position mean absolute error: 1.167677658256888 
position mean square error: 2.9039325467098713 -- position mean absolute error: 1.224426605188847 
position mean square error: 4.224139044287726 -- position mean absolute error: 1.52117

In [4]:
with open('test46k.csv', 'w') as f:
    f.write("beta;dropout;epochs;pos_mse;pos_mae;rot_mse;rot_mae\n")
    for item in results:
        f.write("{};{};{};{};{};{};{}\n".format(item["beta"],item["drop-out"],item["epochs"],item["position squared error"],item["position absolute error"],item["rotation squared error"],item["rotation absolute error"]))

In [3]:
import pandas as pd

with open("data/laser_log/datasetwall.csv", "r") as data:
            laser_db = pd.read_csv(data, delimiter=";")

In [21]:
for i in range(720):
    laser_db.loc[laser_db.index%720 == i, "angle"]= np.radians((i-360)/2)

In [22]:
with open('data/datasetwallmod.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    for item in laser_db.values:
        
        f.write("{};0;{};{};{};{};{}\n".format(item[0],item[2],item[3],item[4],item[5],item[6]))


#Valutation

In [3]:
model.model_cnn.load_weights("models/cp-epoch-train.ckpt")

2022-03-23 14:17:23.462731: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/cp-epoch-train.ckpt: FAILED_PRECONDITION: models/cp-epoch-train.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [4]:
data_test = m.DAO("laser_log/laser_log2203w.csv")

data_test.read()
data_test.divide_data(0.25)

data_tensor_test=m.DAO()

data_tensor_test.TR_features=tf.reshape(data_test.TR_features,[-1,720,1,1])
data_tensor_test.TS_features=tf.reshape(data_test.TS_features,[-1,720,1,1])
data_tensor_test.TR_targets=tf.reshape(data_test.TR_targets,[-1,3])
data_tensor_test.TS_targets=tf.reshape(data_test.TS_targets,[-1,3])



In [26]:
result = model.model_cnn.predict(data_tensor_test.TS_features)
pos_test = result[:,0:2]
rot_test = result[:,2]
pos_real = np.array([data_test.TS_targets["pos_x"].to_list(),data_test.TS_targets["pos_y"].to_list()]).T
rot_real = (data_test.TS_targets["pos_yaw"]).to_numpy()

In [24]:
pos_mae = mean_squared_error(pos_real,pos_test)
rot_mae = mean_absolute_error(rot_real,rot_test)

In [27]:
with open('predict_data2.csv', 'w') as f:
    for i in range(len(rot_real)):
        f.write("{};{};{};{};{};{}\n".format(pos_real[i][0],pos_real[i][1],rot_real[i],pos_test[i][0],pos_test[i][1],rot_test[i]))